## Create Environment
`conda env create -f environment.yml`


`conda activate tcr_metrics`

# Loading Data


In [11]:
#import sys
#sys.path.append("path/to/where/TCR_Metrics/dir/is/for/you")
from TCR_TOOLS.classes.tcr import *
from TCR_TOOLS.core.io import write_pdb
tcr_pdb="/workspaces/Graphormer/TCR_Metrics/test"
tcr = TCR(
        input_pdb=f"/workspaces/Graphormer/TCR_Metrics/test/1ao7.pdb",
        traj_path=None,         # or an XTC/DCD if you have one
        contact_cutoff=5.0, #can keep default, determines distance in Å that constitutes contact
        min_contacts=50, #can keep defaultm determines minimum number of contacts between TCR chains to be recognised as paired chains
        legacy_anarci=True) #if true, prioritise legacy anarci, and only run new anarci if legacy mode fails
print(f"Found {len(tcr.pairs)} pair(s).")
pv = tcr.pairs[0]   # TCRPairView for first pair
#write renumbered file:
write_pdb(os.path.join(tcr_pdb,"1ao7_imgt.pdb"),tcr.imgt_all_structure)


#write only TCR chains renumbered
write_pdb(os.path.join(tcr_pdb,"1ao7_tcr_imgt.pdb"),pv.full_structure)

#write only renumbered variable fragment
write_pdb(os.path.join(tcr_pdb,"1ao7_tcr_vf_imgt.pdb"),pv.variable_structure)


#write only cdr3 beta for example
write_pdb(os.path.join(tcr_pdb,"1ao7_tcr_cdr3b_imgt.pdb"),pv.domain_subset(region_names=["B_CDR3"]))

print(pv.cdr_fr_sequences())
cdr_dict=pv.cdr_fr_sequences()
#write to file
with open(os.path.join(tcr_pdb,"cdr_fr_seq.txt"),"w") as f:
    for cdr in cdr_dict:
        f.write(f">{cdr}\n{cdr_dict[cdr]}\n")

if tcr.original_CDR_FR_RANGES is None:
    pass
else:
    with open(os.path.join(tcr_pdb,"annotation_original_resnums.txt"),"w") as f:
        for cdr in tcr.original_CDR_FR_RANGES:
            f.write(f"{cdr}: {tcr.original_CDR_FR_RANGES[cdr]}\n")





KEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMSIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVTTDSWGKLQFGAGTQVVVTPDIQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDKCVLDMRSMDFKSNSAVAWSNKSDFACANAFNNSIIPE
[info] Numbering chain D with length 194
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLIHYSVGAGITDQGEVPNGYNVSRSTTEDFPLRLLSAAPSQTSVYFCASRPGLAGGRPEQYFGPGTRLTVTEDLKNVFPPEVAVFEPSEAEISHTQKATLVCLATGFYPDHVELSWWVNGKEVHSGVCTDPQPLKEQPALNDSRYALSSRLRVSATFWQDPRNHFRCQVQFYGLSENDEWTQDRAKPVTQIVSAEAWGRAD
[info] Numbering chain E with length 245
Matched TCR pair: D-E | None
A_FR1: (3, 26)
A_CDR1: (27, 38)
A_FR2: (39, 55)
A_CDR2: (56, 65)
A_FR3: (66, 104)
A_CDR3: (105, 117)
A_FR4: (118, 125)
B_FR1: (3, 26)
B_CDR1: (27, 38)
B_FR2: (39, 55)
B_CDR2: (56, 65)
B_FR3: (66, 104)
B_CDR3: (105, 117)
B_FR4: (118, 125)
IMGT renumbering applied.
Found 1 pair(s).
{'A_FR1': 'EVEQNSGPLSVPEGAIASLNCTYS', 'A_CDR1': 'DRGSQS', 'A_FR2': 'FFWYRQYSGKSPELIMS', 'A_CDR2': 'IYSNGD', 'A_FR3': 'KEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLC', 'A_CDR3': 'A

## Aligning Structures

In [14]:
from TCR_TOOLS.classes.tcr_pMHC import TCRpMHC
from TCR_TOOLS.classes.tcr import TCR
from TCR_TOOLS.aligners.structural_fitting import CLASSIC_TCR_ALIGNMENTS,align_and_score_structures


#a = TCRpMHC("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1j8h.pdb", MHC_a_chain_id="A", MHC_b_chain_id="B", Peptide_chain_id="C") #if you have complexes laod like this
#b = TCRpMHC("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1fyt.pdb", MHC_a_chain_id="A", MHC_b_chain_id="B", Peptide_chain_id="C")
#c = TCRpMHC("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1g6r.pdb", MHC_a_chain_id="A", MHC_b_chain_id="B", Peptide_chain_id="C")

a = TCR("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1j8h.pdb").pairs[0]
b = TCR("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1fyt.pdb").pairs[0]
c = TCR("/mnt/larry/lilian/DATA/TCR3d_datasets/TCR_complexes/1g6r.pdb").pairs[0]
#or, if you have a whole directory do:
#tcr_dir="PATH/TO/YOUR/TCR/STRUCTURES"
#TCR_list=[TCR(os.join(tcr_dir, tcr_pdb_name)).pairs[0] for tcr_pdb_name in os.listdir(tcr_dir)]


input_list = [a, b, c]
out_dir="test_aligners_full_run" #visualisations, pdbs of the aligned structures and csv of the results will be saved here if not None
atom_mode=["N", "CA", "C", "O"]  # backbone, can also do "all"
alignment_method="iterative_mediod"
tm_mode="fixed"                 # strongly recommended for non-CA atom sets
make_pymol_viz=True
CLASSIC_TCR_ALIGNMENTS(input_list, out_dir, atom_mode, alignment_method, tm_mode,make_pymol_viz)


#If you want a specific alignment you can also use this:
results_df, outputs=align_and_score_structures(
    structures=input_list,
    align_sel={"A": {"range": (3,10), "ids":[18,20]}, "B": {"ranges": [(1,4), (5,10)]}},  #THESE ARE NONSENSE RANGES/IDS JUST FOR TESTING
    score_sel={"A": {"range": (5,10), "ids":[18,22]}},
    out_dir="test_aligners",
    atom_mode="all",
    alignment_method= "align_to_first",  # "align_to_first" | "mediod" | "iterative_mediod"
    tm_mode = "fixed",                    # "fixed" | "tmalign"
    tm_reduce = "max",                    # if tm_mode="tmalign": "max" | "mean" | "chain1" | "chain2"
    write_aligned_pdbs= True,
    make_pymol_viz = True)


KEEHVIIQAEFYLNPDQSGEFMFDFDGDEIFHVDMAKKETVWRLEEFGRFASFEAQGALANIAVDKANLEIMTKRSNYTPIT
[info] Numbering chain A with length 82
[warning] Chain A could not be numbered with legacy anarci
PKYVKQNTLKLAT
[info] Numbering chain C with length 13
[warning] Chain C could not be numbered with legacy anarci
TRPRFLEQVKHECHFFNGTERVRFLDRYFYHQEEYVRFDSDVGEYRAVTELGRPDAEYWNSQKDLLEQKRAAVDTYCRHNYGVGESFTVQR
[info] Numbering chain B with length 91
[warning] Chain B could not be numbered with legacy anarci
QSVTQLGSHVSVSEGALVLLRCNYSSSVPPYLFWYVQYPNQGLQLLLKYTSAATLVKGINGFEAEFKKSETSFHLTKPSAHMSDAAEYFCAVSESPFGNEKLTFGTGTRLTIIP
[info] Numbering chain D with length 114
KVTQSSRYLVKRTGEKVFLECVQDMDHENMFWYRQDPGLGLRLIYFSYDVKMKEKGDIPEGYSVSREKKERFSLILESASTNQTSMYLCASSSTGLPYGYTFGSGTRLTVV
[info] Numbering chain E with length 111
Matched TCR pair: D-E | None
A_FR1: (3, 26)
A_CDR1: (27, 38)
A_FR2: (39, 55)
A_CDR2: (56, 65)
A_FR3: (66, 104)
A_CDR3: (105, 117)
A_FR4: (118, 125)
B_FR1: (3, 26)
B_CDR1: (27, 38)
B_FR2: (39, 55)
B_CDR2: (